In [29]:
from pyspark.sql import SparkSession

In [30]:
spark = SparkSession.builder.appName('ml').getOrCreate()

In [31]:
training = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [32]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [33]:
from pyspark.ml.feature import VectorAssembler

feature_assembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol= 'Independent Features') #case sensitive col names

we have to group the input columns in form of list

In [34]:
out = feature_assembler.transform(training)

In [35]:
out.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



seelecting featuerea

In [36]:
final_data = out.select('Independent Features', 'Salary')

In [37]:
from pyspark.ml.regression import LinearRegression
# train test splitting
train_data,test_data=final_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

23/11/06 15:33:33 WARN Instrumentation: [6b29c0c7] regParam is zero, which might cause numerical instability and overfitting.


In [38]:
### Coefficients
regressor.coefficients

DenseVector([-90.9091, 1272.7273])

In [39]:
### Intercepts
regressor.intercept

17545.454545455385

In [40]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [41]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16909.090909091006|
|          [24.0,3.0]| 20000|19181.818181818242|
|         [31.0,10.0]| 30000|27454.545454545525|
+--------------------+------+------------------+



In [42]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1757.5757575757464, 3597796.1432506596)